In [1]:
import pandas as pd
import numpy as np

# DATA EXTRACTION

In [2]:
df=pd.read_excel("/content/Input.xlsx")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL_ID  100 non-null    object
 1   URL     100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


In [4]:
df.head(5)

,URL_ID,URL
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...


In [5]:
!unzip /content/MasterDictionary-20240611T101912Z-001.zip -d /content

Archive:  /content/MasterDictionary-20240611T101912Z-001.zip
  inflating: /content/MasterDictionary/positive-words.txt  
  inflating: /content/MasterDictionary/negative-words.txt  


In [6]:
!unzip /content/StopWords-20240611T095359Z-001.zip -d /content

Archive:  /content/StopWords-20240611T095359Z-001.zip
  inflating: /content/StopWords/StopWords_Geographic.txt  
  inflating: /content/StopWords/StopWords_Currencies.txt  
  inflating: /content/StopWords/StopWords_GenericLong.txt  
  inflating: /content/StopWords/StopWords_Generic.txt  
  inflating: /content/StopWords/StopWords_DatesandNumbers.txt  
  inflating: /content/StopWords/StopWords_Auditor.txt  
  inflating: /content/StopWords/StopWords_Names.txt  


In [7]:
from bs4 import BeautifulSoup
import requests

In [8]:
for i in df['URL']:
  response = requests.get(i)
  soup = BeautifulSoup(response.content, 'html.parser')
  formatted_html=soup.prettify()

## Includes content from h1 (for title), paragraph (p) with and without strong tags (<strong>),

In [9]:


def extract_text(formatted_html):

  soup = BeautifulSoup(formatted_html, 'html.parser')

  # Identify header and footer elements (replace with your identification logic)
  header = soup.find('header')
  footer = soup.find('footer')

  # Remove header and footer (if found)
  if header:
    header.decompose()
  if footer:
    footer.decompose()



  # Define tags to extract text from (excluding div and li)
  text_tags = ['title', 'p']

  # Extract title from h1 tag (if found)
  title = ""
  h1_tag = soup.find('h1')
  if h1_tag:
    title = h1_tag.get_text(strip=True).strip()

  # Extract text content from paragraphs, including with or without strong tags
  extracted_text = []
  for tag in text_tags:
    elements = soup.find_all(tag)
    for element in elements:
      element_text = element.get_text(strip=True).strip()
      if tag == 'p':
        # Combine text with strong tags (already handled)
        strong_text = []
        for strong_tag in element.find_all('strong'):
          strong_text.append(strong_tag.get_text(strip=True).strip())
        if strong_text:
          element_text += '\n' + '\n'.join(strong_text)

      # No need for separate logic for text without strong tags
      if element_text:
        extracted_text.append(element_text)

  # Combine title and extracted text (combines text with and without strong tags)
  combined_text = [title] + extracted_text

  return '\n'.join(combined_text)


# Assuming your DataFrame is called 'df' and the URL column is named 'url'
for i,url in enumerate(df['URL']):
  response = requests.get(url)
  if response.status_code == 200:
    formatted_html = response.content.decode('utf-8')  # Decode for proper parsing
    extracted_text = extract_text(formatted_html)
    df.loc[i, 'Extracted Text'] = extracted_text
  else:
    print(f"URL: {url}, Error: {response.status_code}")


URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/, Error: 404
URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/, Error: 404


In [55]:
output_file = 'extracted_texts_with_links.txt'

# Write the URLs and the extracted texts to the file
with open(output_file, 'w', encoding='utf-8') as file:
    for i, row in df.iterrows():
        url = row['URL']
        text = row['Extracted Text']
        file.write(f"URL: {url}\n{text}\n\n")  # URL followed by extracted text, double new line for separation

print(f"URLs and extracted texts have been written to {output_file}")


URLs and extracted texts have been written to extracted_texts_with_links.txt


In [10]:
df.head()

,URL_ID,URL,Extracted Text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...


In [11]:
df = df.dropna(subset=['Extracted Text'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   URL_ID          98 non-null     object
 1   URL             98 non-null     object
 2   Extracted Text  98 non-null     object
dtypes: object(3)
memory usage: 3.1+ KB


# DATA CLEANING
### Removing stopwords and lemmatizing text

lmmatizing= "running", "ran", and "runs" can all be lemmatized to their base form "run".


In [13]:
import zipfile
import re
import pandas as pd
import chardet
from nltk.stem import PorterStemmer
import nltk
nltk.download('punkt')
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [56]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lem=WordNetLemmatizer()


In [15]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [16]:
stopword_auditor=open('/content/StopWords/StopWords_Auditor.txt','r',encoding='ISO-8859-1')
stopwords_currencies=open('/content/StopWords/StopWords_Currencies.txt','r',encoding='ISO-8859-1')
stopword_DatesandNumbers=open('/content/StopWords/StopWords_DatesandNumbers.txt','r',encoding='ISO-8859-1')
stopword_Generic=open('/content/StopWords/StopWords_Generic.txt','r',encoding='ISO-8859-1')
stopword_genericLong=open('/content/StopWords/StopWords_GenericLong.txt','r',encoding='ISO-8859-1')
stopword_geographic=open('/content/StopWords/StopWords_Geographic.txt','r',encoding='ISO-8859-1')
stopword_Names=open('/content/StopWords/StopWords_Names.txt','r',encoding='ISO-8859-1')

In [17]:
def text_corpus(x):
  string_format=str(x).lower()
  lower_words=re.sub('[^a-zA-Z]+',' ',string_format).strip()
  token=word_tokenize(lower_words)
  token_word=[t for t in token if t not in(stopword_auditor,stopwords_currencies,stopword_DatesandNumbers,stopword_Generic,stopword_genericLong,stopword_geographic,stopword_Names)]
  lemantizzed=[lem.lemmatize(w) for w in token_word]
  return lemantizzed

In [18]:
def text_corpus1(x):
  string_format=str(x).lower()
  token=word_tokenize(string_format)
  token_word=[t for t in token if t not in(stopword_auditor,stopwords_currencies,stopword_DatesandNumbers,stopword_Generic,stopword_genericLong,stopword_geographic,stopword_Names)]
  lemantizzed=[lem.lemmatize(w) for w in token_word]
  return lemantizzed

In [19]:
df['preprocessed_words']=df['Extracted Text'].apply(text_corpus)

In [20]:
df['cleaned_text']=df['Extracted Text'].apply(text_corpus1)

In [21]:
df.head()

,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e..."
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the..."
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica..."
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up..."
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente..."


# SENTIMENT ANALYSIS
Calculating the given term

In [22]:
file_neg=open('/content/MasterDictionary/negative-words.txt','r',encoding='ISO-8859-1')
file_neg.seek(0)
neg_split=file_neg.read().split()
len(neg_split)

4783

In [23]:
file_pos=open('/content/MasterDictionary/positive-words.txt','r',encoding='ISO-8859-1')
file_pos.seek(0)
pos_split=file_pos.read().split()
len(pos_split)

2006

# 1)NEGATIVE SCORE

In [24]:
df['NEGATIVE SCORE'] = df['preprocessed_words'].apply(lambda x: sum(1 for word in x if word in neg_split))


In [25]:
df['NEGATIVE SCORE']

0      2
1     32
2     30
3     87
4     10
      ..
95    65
96    37
97     1
98     4
99    60
Name: NEGATIVE SCORE, Length: 98, dtype: int64

# 2)POSITIVE SCORE

In [26]:
df['POSITIVE SCORE'] = df['preprocessed_words'].apply(lambda x: sum(1 for word in x if word in pos_split))

In [27]:
df['POSITIVE SCORE']

0     18
1     74
2     53
3     51
4     33
      ..
95    40
96    41
97    12
98    33
99    47
Name: POSITIVE SCORE, Length: 98, dtype: int64

In [28]:
df.head()

,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text,NEGATIVE SCORE,POSITIVE SCORE
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e...",2,18
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the...",32,74
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica...",30,53
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up...",87,51
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente...",10,33


# 3)Polarity Score
# 4)Subjectivity Score

In [31]:
df['total_words'] = df['preprocessed_words'].apply(len)

# Calculate Polarity Score
df['POLARITY SCORE'] = (df['POSITIVE SCORE'] - df['NEGATIVE SCORE']) / (
    (df['NEGATIVE SCORE'] + df['NEGATIVE SCORE']) + 0.000001
)

# Calculate Subjectivity Score
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE']) / (
    df['total_words'] + 0.000001
)

In [32]:
df.head()

,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text,NEGATIVE SCORE,POSITIVE SCORE,total_words,POLARITY SCORE,SUBJECTIVITY SCORE
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e...",2,18,572,3.999999,0.034965
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the...",32,74,1652,0.656250,0.064165
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica...",30,53,1312,0.383333,0.063262
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up...",87,51,1306,-0.206897,0.105666
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente...",10,33,902,1.150000,0.047672


In [33]:
print(df['Extracted Text'].iloc[0])


Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040. - Blackcoffer Insights
Efficient Supply Chain Assessment: Overcoming Technical Hurdles for Web Application Development
Streamlined Integration: Interactive Brokers API with Python for Desktop Trading Application
Efficient Data Integration and User-Friendly Interface Development: Navigating Challenges in Web Application Deployment
Effective Management of Social Media Data Extraction: Strategies for Authentication, Security, and Reliability
AI Bot Audio to audio
Methodology for ETL Discovery Tool using LLMA, OpenAI, Langchain
Methodology for database discovery tool using openai, LLMA, Langchain
Chatbot using VoiceFlow
Rising IT cities and its impact on the economy, environment, infrastructure, and city life by the year 2040.
Rising IT Cities and Their Impact on the Economy, Environment, Infrastructure, and City Life in Future
Internet Demand’s Evolution, Communication Impact, and

### *   punkt: Use for splitting text into sentences and words.
   ### *   cmudict: Used for syllable analysis(which provides a mapping between words and their pronunciations)








In [34]:
import nltk
nltk.download('punkt')
nltk.download('cmudict')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


True

# 5)AVERAGE SENTENCE LENGTH  
# 6) PERCENTAGE OF COMPLEX WORDS

### * sent_tokenize: Splits text into sentences.
### * word_tokenize: Splits text into words.

In [35]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import cmudict

# Ensure necessary NLTK data is downloaded


# Load CMU Pronouncing Dictionary
d = cmudict.dict()

# Function to count syllables in a word using CMU Pronouncing Dictionary
def count_syllables(word):
    if word.lower() in d:
        return [len(list(y for y in x if y[-1].isdigit())) for x in d[word.lower()]][0]
    else:
        return sum([1 for char in word if char in "aeiouAEIOU"])  # Fallback to vowel counting

# Function to count complex words (words with three or more syllables)
def is_complex(word):
    return count_syllables(word) >= 3

# Function to calculate readability metrics
def calculate_readability_metrics(text):
    sentences = sent_tokenize(text)
    words = word_tokenize(text)

    num_sentences = len(sentences)
    num_words = len(words)
    num_complex_words = sum([1 for word in words if is_complex(word)])

    asl = num_words / num_sentences if num_sentences > 0 else 0
    pcw = num_complex_words / num_words if num_words > 0 else 0

    fog_index = 0.4 * (asl + pcw * 100)  # Multiplying PCW by 100 to match percentage calculation

    return asl, pcw * 100, fog_index

# Assuming df is the DataFrame containing the text data
df['AVERAGE SENTENCE LENGTH'], df['PERCENTAGE OF COMPLEX WORDS '], df['FOG INDEX'] = zip(*df['Extracted Text'].apply(calculate_readability_metrics))

# Display the DataFrame with the calculated readability metrics
print(df)


             URL_ID                                                URL  \
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...   
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...   
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...   
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...   
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...   
..              ...                                                ...   
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...   
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...   
97  blackassign0098  https://insights.blackcoffer.com/contribution-...   
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...   
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...   

                                       Extracted Text  \
0   \nRising IT cities and its impact on the econo... 



# 7)   AVERAGE NUMBER OF WORDS PER SENTENCE
# 8)  COMPLEX WORD COUNT'



In [36]:
# Helper function to count syllables in a word
def count_syllables(word):
    if not word:  # check for empty string
        return 0
    vowels = "aeiouy"
    word = word.lower().strip(".:;?!")
    count = 0
    if word and word[0] in vowels:  # check if word is not empty and starts with a vowel
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

# Function to calculate the average number of words per sentence
def average_words_per_sentence(text):
    if not text:  # handle empty text
        return 0
    words = word_tokenize(text)
    sentences = sent_tokenize(text)
    total_words = len(words)
    total_sentences = len(sentences)
    return total_words / total_sentences if total_sentences > 0 else 0

# Function to count the number of complex words
def complex_word_count(text):
    if not text:  # handle empty text
        return 0
    words = word_tokenize(text)
    complex_words = [word for word in words if count_syllables(word) > 2]
    return len(complex_words)

In [37]:
df['AVERAGE NUMBER OF WORDS PER SENTENCE'] = df['Extracted Text'].apply(average_words_per_sentence)

In [38]:
df['COMPLEX WORD COUNT'] = df['Extracted Text'].apply(complex_word_count)

In [39]:
print(df)

             URL_ID                                                URL  \
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...   
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...   
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...   
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...   
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...   
..              ...                                                ...   
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...   
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...   
97  blackassign0098  https://insights.blackcoffer.com/contribution-...   
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...   
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...   

                                       Extracted Text  \
0   \nRising IT cities and its impact on the econo... 

In [40]:
df['Count Words']=df['total_words']

In [41]:
df.drop(['total_words'],axis=1,inplace=True)

In [42]:
df

,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text,NEGATIVE SCORE,POSITIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,Count Words
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e...",2,18,3.999999,0.034965,21.633333,20.030817,16.665660,21.633333,123,572
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the...",32,74,0.656250,0.064165,23.612500,23.292747,18.762099,23.612500,421,1652
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica...",30,53,0.383333,0.063262,24.327869,29.986523,21.725757,24.327869,423,1312
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up...",87,51,-0.206897,0.105666,26.964286,27.880795,21.938032,26.964286,414,1306
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente...",10,33,1.150000,0.047672,22.727273,23.000000,18.290909,22.727273,219,902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,\nDue to the COVID-19 the repercussion of the ...,"[due, to, the, covid, the, repercussion, of, t...","[due, to, the, covid-19, the, repercussion, of...",65,40,-0.192308,0.078829,26.814815,22.099448,19.565705,26.814815,324,1332
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,\nImpact of COVID-19 pandemic on office space ...,"[impact, of, covid, pandemic, on, office, spac...","[impact, of, covid-19, pandemic, on, office, s...",37,41,0.054054,0.060278,33.511628,16.655101,20.066691,33.511628,220,1294
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,\nContribution of handicrafts (Visual Arts & L...,"[contribution, of, handicraft, visual, art, li...","[contribution, of, handicraft, (, visual, art,...",1,12,5.499997,0.035714,47.000000,28.605201,30.242080,47.000000,102,364
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,How COVID-19 is impacting payment preferences?...,"[how, covid, is, impacting, payment, preferenc...","[how, covid-19, is, impacting, payment, prefer...",4,33,3.625000,0.049598,24.970588,18.256773,17.290944,24.970588,141,746


#9) WORD COUNT

In [43]:

# Function to count words
def count_words(words):
  return len(words)

# Apply the function to the 'text_column' and add a new column 'word_count'
df['WORD COUNT'] = df['preprocessed_words'].apply(count_words)

# Print the DataFrame with the new 'word_count' column
print(df)


             URL_ID                                                URL  \
0   blackassign0001  https://insights.blackcoffer.com/rising-it-cit...   
1   blackassign0002  https://insights.blackcoffer.com/rising-it-cit...   
2   blackassign0003  https://insights.blackcoffer.com/internet-dema...   
3   blackassign0004  https://insights.blackcoffer.com/rise-of-cyber...   
4   blackassign0005  https://insights.blackcoffer.com/ott-platform-...   
..              ...                                                ...   
95  blackassign0096  https://insights.blackcoffer.com/what-is-the-r...   
96  blackassign0097  https://insights.blackcoffer.com/impact-of-cov...   
97  blackassign0098  https://insights.blackcoffer.com/contribution-...   
98  blackassign0099  https://insights.blackcoffer.com/how-covid-19-...   
99  blackassign0100  https://insights.blackcoffer.com/how-will-covi...   

                                       Extracted Text  \
0   \nRising IT cities and its impact on the econo... 

In [57]:
df


,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text,NEGATIVE SCORE,POSITIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,Count Words,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVERAGE WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e...",2,18,3.999999,0.034965,21.633333,20.030817,16.665660,21.633333,123,572,572,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",5,5.062937
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the...",32,74,0.656250,0.064165,23.612500,23.292747,18.762099,23.612500,421,1652,1652,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",4,5.443705
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica...",30,53,0.383333,0.063262,24.327869,29.986523,21.725757,24.327869,423,1312,1312,"[3, 2, 1, 4, 5, 2, 1, 1, 5, 2, 3, 2, 3, 2, 1, ...",13,5.942835
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up...",87,51,-0.206897,0.105666,26.964286,27.880795,21.938032,26.964286,414,1306,1306,"[2, 1, 4, 1, 1, 2, 1, 3, 3, 3, 2, 3, 2, 1, 3, ...",6,5.853752
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente...",10,33,1.150000,0.047672,22.727273,23.000000,18.290909,22.727273,219,902,902,"[1, 2, 1, 1, 2, 1, 1, 4, 3, 1, 3, 3, 2, 3, 2, ...",7,5.454545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,\nDue to the COVID-19 the repercussion of the ...,"[due, to, the, covid, the, repercussion, of, t...","[due, to, the, covid-19, the, repercussion, of...",65,40,-0.192308,0.078829,26.814815,22.099448,19.565705,26.814815,324,1332,1332,"[1, 1, 1, 2, 1, 4, 1, 1, 4, 3, 2, 3, 2, 1, 3, ...",4,5.228228
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,\nImpact of COVID-19 pandemic on office space ...,"[impact, of, covid, pandemic, on, office, spac...","[impact, of, covid-19, pandemic, on, office, s...",37,41,0.054054,0.060278,33.511628,16.655101,20.066691,33.511628,220,1294,1294,"[2, 1, 2, 3, 1, 3, 2, 1, 1, 2, 3, 3, 2, 3, 2, ...",7,4.783617
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,\nContribution of handicrafts (Visual Arts & L...,"[contribution, of, handicraft, visual, art, li...","[contribution, of, handicraft, (, visual, art,...",1,12,5.499997,0.035714,47.000000,28.605201,30.242080,47.000000,102,364,364,"[4, 1, 3, 2, 1, 5, 1, 1, 2, 4, 3, 2, 3, 2, 1, ...",1,5.695055
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,How COVID-19 is impacting payment preferences?...,"[how, covid, is, impacting, payment, preferenc...","[how, covid-19, is, impacting, payment, prefer...",4,33,3.625000,0.049598,24.970588,18.256773,17.290944,24.970588,141,746,746,"[1, 2, 1, 3, 2, 4, 1, 2, 1, 3, 2, 4, 3, 2, 3, ...",5,5.046917


In [58]:
df.drop(['Count Words'],axis=1,inplace=True)

# 10)SYLLABLE PER WORD

In [47]:
def count_syllables(word):
    vowels = "aeiouy"
    count = 0
    prev_char_was_vowel = False
    for char in word:
        if char.lower() in vowels:
            if not prev_char_was_vowel:
                count += 1
            prev_char_was_vowel = True
        else:
            prev_char_was_vowel = False

    # Handling exceptions
    if word.endswith(("es", "ed")):
        count -= 1

    # Minimum syllable count is 1
    return max(count, 1)

# Apply function to each word in DataFrame
def count_syllables_in_list(word_list):
    return [count_syllables(word) for word in word_list]




In [48]:
df['SYLLABLE PER WORD'] = df['preprocessed_words'].apply(count_syllables_in_list)

# 11)PERSONAL PRONOUNS

In [49]:
pattern = r'\b(?:I|we|my|ours|us)\b'

# Function to count personal pronouns
def count_personal_pronouns(word_list):
    text = ' '.join(word_list)
    return len(re.findall(pattern, text, flags=re.IGNORECASE))

# Apply function to each row in DataFrame
df['PERSONAL PRONOUNS'] = df['preprocessed_words'].apply(count_personal_pronouns)


In [50]:
df

,URL_ID,URL,Extracted Text,preprocessed_words,cleaned_text,NEGATIVE SCORE,POSITIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,Count Words,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT cities and its impact on the econo...,"[rising, it, city, and, it, impact, on, the, e...","[rising, it, city, and, it, impact, on, the, e...",2,18,3.999999,0.034965,21.633333,20.030817,16.665660,21.633333,123,572,572,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",5
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,\nRising IT Cities and Their Impact on the Eco...,"[rising, it, city, and, their, impact, on, the...","[rising, it, city, and, their, impact, on, the...",32,74,0.656250,0.064165,23.612500,23.292747,18.762099,23.612500,421,1652,1652,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",4
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,"\nInternet Demand's Evolution, Communication I...","[internet, demand, s, evolution, communication...","[internet, demand, 's, evolution, ,, communica...",30,53,0.383333,0.063262,24.327869,29.986523,21.725757,24.327869,423,1312,1312,"[3, 2, 1, 4, 5, 2, 1, 1, 5, 2, 3, 2, 3, 2, 1, ...",13
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,\nRise of Cybercrime and its Effect in upcomin...,"[rise, of, cybercrime, and, it, effect, in, up...","[rise, of, cybercrime, and, it, effect, in, up...",87,51,-0.206897,0.105666,26.964286,27.880795,21.938032,26.964286,414,1306,1306,"[2, 1, 4, 1, 1, 2, 1, 3, 3, 3, 2, 3, 2, 1, 3, ...",6
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,\nOTT platform and its impact on the entertain...,"[ott, platform, and, it, impact, on, the, ente...","[ott, platform, and, it, impact, on, the, ente...",10,33,1.150000,0.047672,22.727273,23.000000,18.290909,22.727273,219,902,902,"[1, 2, 1, 1, 2, 1, 1, 4, 3, 1, 3, 3, 2, 3, 2, ...",7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,\nDue to the COVID-19 the repercussion of the ...,"[due, to, the, covid, the, repercussion, of, t...","[due, to, the, covid-19, the, repercussion, of...",65,40,-0.192308,0.078829,26.814815,22.099448,19.565705,26.814815,324,1332,1332,"[1, 1, 1, 2, 1, 4, 1, 1, 4, 3, 2, 3, 2, 1, 3, ...",4
96,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,\nImpact of COVID-19 pandemic on office space ...,"[impact, of, covid, pandemic, on, office, spac...","[impact, of, covid-19, pandemic, on, office, s...",37,41,0.054054,0.060278,33.511628,16.655101,20.066691,33.511628,220,1294,1294,"[2, 1, 2, 3, 1, 3, 2, 1, 1, 2, 3, 3, 2, 3, 2, ...",7
97,blackassign0098,https://insights.blackcoffer.com/contribution-...,\nContribution of handicrafts (Visual Arts & L...,"[contribution, of, handicraft, visual, art, li...","[contribution, of, handicraft, (, visual, art,...",1,12,5.499997,0.035714,47.000000,28.605201,30.242080,47.000000,102,364,364,"[4, 1, 3, 2, 1, 5, 1, 1, 2, 4, 3, 2, 3, 2, 1, ...",1
98,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,How COVID-19 is impacting payment preferences?...,"[how, covid, is, impacting, payment, preferenc...","[how, covid-19, is, impacting, payment, prefer...",4,33,3.625000,0.049598,24.970588,18.256773,17.290944,24.970588,141,746,746,"[1, 2, 1, 3, 2, 4, 1, 2, 1, 3, 2, 4, 3, 2, 3, ...",5


# 12)AVERAGE WORD LENGTH

In [51]:
# Function to calculate average word length
def average_word_length(word_list):
    total_characters = sum(len(word) for word in word_list)
    total_words = len(word_list)
    if total_words > 0:
        return total_characters / total_words
    else:
        return 0



In [52]:
df['AVERAGE WORD LENGTH'] = df['preprocessed_words'].apply(average_word_length)


In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   URL_ID                                98 non-null     object 
 1   URL                                   98 non-null     object 
 2   Extracted Text                        98 non-null     object 
 3   preprocessed_words                    98 non-null     object 
 4   cleaned_text                          98 non-null     object 
 5   NEGATIVE SCORE                        98 non-null     int64  
 6   POSITIVE SCORE                        98 non-null     int64  
 7   POLARITY SCORE                        98 non-null     float64
 8   SUBJECTIVITY SCORE                    98 non-null     float64
 9   AVERAGE SENTENCE LENGTH               98 non-null     float64
 10  PERCENTAGE OF COMPLEX WORDS           98 non-null     float64
 11  FOG INDEX                 

In [60]:
df.drop(['Extracted Text','preprocessed_words','cleaned_text'],axis=1,inplace=True)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 99
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   URL_ID                                98 non-null     object 
 1   URL                                   98 non-null     object 
 2   NEGATIVE SCORE                        98 non-null     int64  
 3   POSITIVE SCORE                        98 non-null     int64  
 4   POLARITY SCORE                        98 non-null     float64
 5   SUBJECTIVITY SCORE                    98 non-null     float64
 6   AVERAGE SENTENCE LENGTH               98 non-null     float64
 7   PERCENTAGE OF COMPLEX WORDS           98 non-null     float64
 8   FOG INDEX                             98 non-null     float64
 9   AVERAGE NUMBER OF WORDS PER SENTENCE  98 non-null     float64
 10  COMPLEX WORD COUNT                    98 non-null     int64  
 11  WORD COUNT                

In [62]:
df.head()

,URL_ID,URL,NEGATIVE SCORE,POSITIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVERAGE SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVERAGE NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVERAGE WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,2,18,3.999999,0.034965,21.633333,20.030817,16.665660,21.633333,123,572,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",5,5.062937
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,32,74,0.656250,0.064165,23.612500,23.292747,18.762099,23.612500,421,1652,"[2, 1, 2, 1, 1, 2, 1, 1, 4, 4, 5, 1, 2, 2, 1, ...",4,5.443705
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,30,53,0.383333,0.063262,24.327869,29.986523,21.725757,24.327869,423,1312,"[3, 2, 1, 4, 5, 2, 1, 1, 5, 2, 3, 2, 3, 2, 1, ...",13,5.942835
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,87,51,-0.206897,0.105666,26.964286,27.880795,21.938032,26.964286,414,1306,"[2, 1, 4, 1, 1, 2, 1, 3, 3, 3, 2, 3, 2, 1, 3, ...",6,5.853752
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,10,33,1.150000,0.047672,22.727273,23.000000,18.290909,22.727273,219,902,"[1, 2, 1, 1, 2, 1, 1, 4, 3, 1, 3, 3, 2, 3, 2, ...",7,5.454545


In [64]:
from IPython.display import FileLink

# Export to Excel format
df.to_excel('output.xlsx', index=False)

# Download the file
FileLink('output.xlsx')

/content/output.xlsx